# scikit-learnとXGBoostの使い方

scikit-learn は、前処理・学習・評価を統一された API で扱えるライブラリです。
XGBoost は勾配ブースティングを高速かつ高精度に実装した手法で、タブularデータで強力な選択肢になります。

このノートでは、まず scikit-learn の基本フローを固め、その上で XGBoost を同じ問題に適用して比較します。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

try:
    from xgboost import XGBClassifier
    XGBOOST_AVAILABLE = True
except ModuleNotFoundError:
    XGBClassifier = None
    XGBOOST_AVAILABLE = False

sns.set_theme(style="whitegrid", context="notebook")


## 1. scikit-learnの基本フロー

ここでは乳がんデータセットを使って、
`データ分割 → パイプライン定義 → 学習 → 評価` を一気通貫で実行します。

評価指標は Accuracy と ROC-AUC を使います。
Accuracy は分類の正解率、ROC-AUC は分類しきい値を動かしたときの識別性能（順位づけの上手さ）を見ます。

In [ ]:
cancer = load_breast_cancer(as_frame=True)
X = cancer.data
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

baseline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=2000, random_state=42)),
])

baseline.fit(X_train, y_train)
pred = baseline.predict(X_test)
proba = baseline.predict_proba(X_test)[:, 1]

print(f"accuracy: {accuracy_score(y_test, pred):.3f}")
print(f"roc_auc : {roc_auc_score(y_test, proba):.3f}")


混同行列を確認すると、単純な精度だけでは見えない誤分類の偏りを把握できます。

In [ ]:
cm = confusion_matrix(y_test, pred)
fig, ax = plt.subplots(figsize=(4.5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False, ax=ax)
ax.set_title("Logistic Regression Confusion Matrix")
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")
plt.show()


## 2. 交差検証とハイパーパラメータ探索

分割1回の評価だけでは偶然の影響が残るので、交差検証で安定性を見ます。
その後、`GridSearchCV` で `C` を探索します。

ここでの流れは次の通りです。
- まず CV で、モデル設定が全体として安定しているかを確認
- 次に訓練データ側で GridSearch を実行し、テストデータは最後まで未使用に保つ

こうすることで、チューニング結果を過度に楽観視しにくくなります。

In [ ]:
cv_scores = cross_val_score(
    baseline,
    X,
    y,
    cv=5,
    scoring="roc_auc",
)
print("CV ROC-AUC (5-fold):", np.round(cv_scores, 4))
print("mean:", cv_scores.mean())


In [ ]:
search = GridSearchCV(
    estimator=Pipeline([
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=2000, random_state=42)),
    ]),
    param_grid={
        "model__C": [0.01, 0.1, 1.0, 3.0, 10.0],
    },
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
)
search.fit(X_train, y_train)

best_lr = search.best_estimator_
best_pred = best_lr.predict(X_test)
best_proba = best_lr.predict_proba(X_test)[:, 1]

print("best params:", search.best_params_)
print(f"tuned accuracy: {accuracy_score(y_test, best_pred):.3f}")
print(f"tuned roc_auc : {roc_auc_score(y_test, best_proba):.3f}")


## 3. XGBoostを同じ問題に適用する

XGBoost が利用可能な環境では、同じ学習データで性能比較します。
XGBoost は木ベースなので、今回の設定では標準化を必須としません。

In [ ]:
if XGBOOST_AVAILABLE:
    xgb = XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        eval_metric="logloss",
        random_state=42,
    )
    xgb.fit(X_train, y_train)

    xgb_pred = xgb.predict(X_test)
    xgb_proba = xgb.predict_proba(X_test)[:, 1]

    print(f"xgb accuracy: {accuracy_score(y_test, xgb_pred):.3f}")
    print(f"xgb roc_auc : {roc_auc_score(y_test, xgb_proba):.3f}")
else:
    print("xgboost がインストールされていないため、このセルはスキップされました。")


XGBoost を使えた場合は、重要特徴量を見て学習がどこに注目したかを確認します。

In [ ]:
if XGBOOST_AVAILABLE:
    importances = pd.Series(xgb.feature_importances_, index=X.columns).sort_values(ascending=False).head(12)
    fig, ax = plt.subplots(figsize=(7, 4.5))
    sns.barplot(x=importances.values, y=importances.index, orient="h", ax=ax)
    ax.set_title("Top 12 Feature Importances (XGBoost)")
    ax.set_xlabel("importance")
    ax.set_ylabel("feature")
    plt.tight_layout()
    plt.show()
else:
    print("xgboost 未導入のため、重要特徴量の表示をスキップしました。")


## 4. 使い分けの実務メモ

- scikit-learn: 実験の土台を作るのが速く、比較実験を回しやすい
- XGBoost: 表形式データで高い精度を狙いやすい

まず scikit-learn で前処理と評価設計を固め、
次に XGBoost を含む複数モデルを同一条件で比較する流れが実践的です。